In [1]:
#make sure it uses the custom baselines package
import sys
sys.path.insert(0,'./baselines/')

import argparse
# coding: utf-8

# Take as input a compressed pretrained network or run T_REX before hand
# Then run MCMC and save posterior chain


import pickle
import copy
import gym
import time
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from run_test import *
from StrippedNet import EmbeddingNet
from baselines.common.trex_utils import preprocess

/home/dsbrown/anaconda3/envs/deeplearning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dsbrown/anaconda3/envs/deeplearning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dsbrown/anaconda3/envs/deeplearning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dsbrown/anaconda3/e

Logging to /tmp/openai-2020-02-04-16-47-01-809131


In [18]:
def generate_mean_map_noop_demos(env, env_name, agent, mean_path, map_path):
    demonstrations = []
    learning_returns = []
    learning_rewards = []
    for model_path in [mean_path, map_path]:

        agent.load(model_path)
        episode_count = 2
        for i in range(episode_count):
            done = False
            traj = []
            gt_rewards = []
            r = 0

            ob = env.reset()
            steps = 0
            acc_reward = 0
            while steps < 8000:
                action = agent.act(ob, r, done)
                ob, r, done, _ = env.step(action)
                if render:
                    env.render()
                ob_processed = preprocess(ob, env_name)
                #ob_processed = ob_processed[0] #get rid of first dimension ob.shape = (1,84,84,4)
                traj.append(ob_processed)

                gt_rewards.append(r[0])
                steps += 1
                acc_reward += r[0]
                if done:
                    
                    break
            print("checkpoint: {}, steps: {}, return: {}".format(model_path, steps,acc_reward))
            print("traj length", len(traj))
            print("demo length", len(demonstrations))
            
            
            #check if it went too long and segment it into good and bad
            if steps == 8000:
                traj_rewards = gt_rewards
                for i in range(len(traj)-1,-1,-1):
                    if traj_rewards[i] > 0:
                        indx = i+1
                        print(indx, "last time when scored")
                        break
                print("total return after", np.sum(traj_rewards[indx:]))
                good_segment = traj[:indx+1]
                good_rewards = traj_rewards[:indx+1]
                bad_segment = traj[indx+1:]
                bad_rewards = traj_rewards[indx+1:]
                print("good segment length", len(good_segment), "reward", np.sum(good_rewards))
                print("bad segment length", len(bad_segment), "rewards", np.sum(bad_rewards))
            
                print("adding to demonstrations")
                demonstrations.append(good_segment)
                learning_returns.append(np.sum(good_rewards))
                learning_rewards.append(good_rewards)
                demonstrations.append(bad_segment)
                learning_returns.append(np.sum(bad_rewards))
                learning_rewards.append(bad_rewards)
            else:
                demonstrations.append(traj)
                learning_returns.append(acc_reward)
                learning_rewards.append(gt_rewards)

    #add no-op demos
    done = False
    traj = []
    gt_rewards = []
    r = 0

    ob = env.reset()
    steps = 0
    acc_reward = 0
    while steps < 5000:
        action = 0#agent.act(ob, r, done)
        ob, r, done, _ = env.step(action)
        ob_processed = preprocess(ob, env_name)
        #ob_processed = ob_processed[0] #get rid of first dimension ob.shape = (1,84,84,4)
        traj.append(ob_processed)

        gt_rewards.append(r[0])
        steps += 1
        acc_reward += r[0]
        if done:
            print("checkpoint: {}, steps: {}, return: {}".format("noop", steps,acc_reward))
            break
    print("noop traj length", len(traj))
    print("demo length", len(demonstrations))
    demonstrations.append(traj)
    learning_returns.append(acc_reward)
    learning_rewards.append(gt_rewards)


    return demonstrations, learning_returns, learning_rewards



def generate_novice_demos(env, env_name, agent, model_dir):
    checkpoint_min = 50
    checkpoint_max = 600
    checkpoint_step = 50
    checkpoints = []
    if env_name == "enduro":
        checkpoint_min = 3100
        checkpoint_max = 3650
    elif env_name == "seaquest":
        checkpoint_min = 10
        checkpoint_max = 65
        checkpoint_step = 5
    for i in range(checkpoint_min, checkpoint_max + checkpoint_step, checkpoint_step):
        if i < 10:
            checkpoints.append('0000' + str(i))
        elif i < 100:
            checkpoints.append('000' + str(i))
        elif i < 1000:
            checkpoints.append('00' + str(i))
        elif i < 10000:
            checkpoints.append('0' + str(i))
    print(checkpoints)



    demonstrations = []
    learning_returns = []
    learning_rewards = []
    for checkpoint in checkpoints:

        model_path = model_dir + env_name + "_25/" + checkpoint
        if env_name == "seaquest":
            model_path = model_dir + env_name + "_5/" + checkpoint

        agent.load(model_path)
        episode_count = 1
        for i in range(episode_count):
            done = False
            traj = []
            gt_rewards = []
            r = 0

            ob = env.reset()
            steps = 0
            acc_reward = 0
            while True:
                action = agent.act(ob, r, done)
                ob, r, done, _ = env.step(action)
                ob_processed = preprocess(ob, env_name)
                #ob_processed = ob_processed[0] #get rid of first dimension ob.shape = (1,84,84,4)
                traj.append(ob_processed)

                gt_rewards.append(r[0])
                steps += 1
                acc_reward += r[0]
                if done:
                    print("checkpoint: {}, steps: {}, return: {}".format(checkpoint, steps,acc_reward))
                    break
            print("traj length", len(traj))
            print("demo length", len(demonstrations))
            demonstrations.append(traj)
            learning_returns.append(acc_reward)
            learning_rewards.append(gt_rewards)

    return demonstrations, learning_returns, learning_rewards



def create_mcmc_likelihood_data(demonstrations):
    '''create all pairwise rankings given list of sorted demonstrations'''
    training_obs = []
    training_labels = []
    num_demos = len(demonstrations)
    for i in range(num_demos):
        for j in range(i+1,num_demos):
            #print(i,j)
            traj_i = demonstrations[i]
            traj_j = demonstrations[j]
            label = 1
            training_obs.append((traj_i, traj_j))
            training_labels.append(label)

    return training_obs, training_labels









def predict_reward_sequence(net, traj):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    rewards_from_obs = []
    with torch.no_grad():
        for s in traj:
            r = net.cum_return(torch.from_numpy(np.array(s)).float().to(device)).item()
            rewards_from_obs.append(r)
    return rewards_from_obs

def predict_traj_return(net, traj):
    return sum(predict_reward_sequence(net, traj))

def print_traj_returns(reward_net, demonstrations):
    #print out predicted cumulative returns and actual returns
    with torch.no_grad():
        pred_returns = [predict_traj_return(reward_net, traj) for traj in demonstrations]
    for i, p in enumerate(pred_returns):
        print(i,p,sorted_returns[i])

def calc_linearized_pairwise_ranking_loss(last_layer, pairwise_prefs, demo_cnts, confidence=1):
    '''use (i,j) indices and precomputed feature counts to do faster pairwise ranking loss'''
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # Assume that we are on a CUDA machine, then this should print a CUDA device:
    #print(device)
    #don't need any gradients
    with torch.no_grad():

        #do matrix multiply with last layer of network and the demo_cnts
        #print(list(reward_net.fc2.parameters()))
        linear = last_layer.weight.data  #not using bias
        #print(linear)
        #print(bias)
        weights = linear.squeeze() #append bias and weights from last fc layer together
        #print('weights',weights)
        #print('demo_cnts', demo_cnts)
        demo_returns = confidence * torch.mv(demo_cnts, weights)

        #positivity prior
        if demo_returns[0] < 0.0:
            return torch.Tensor([-float("Inf")])


        loss_criterion = nn.CrossEntropyLoss(reduction='sum') #sum up losses
        cum_log_likelihood = 0.0
        outputs = torch.zeros(len(pairwise_prefs),2) #each row is a new pair of returns
        for p, ppref in enumerate(pairwise_prefs):
            i,j = ppref
            outputs[p,:] = torch.tensor([demo_returns[i], demo_returns[j]])
        labels = torch.ones(len(pairwise_prefs)).long()

        #outputs = outputs.unsqueeze(0)
        #print(outputs)
        #print(labels)
        cum_log_likelihood = -loss_criterion(outputs, labels)
            #if labels == 0:
            #    log_likelihood = torch.log(return_i/(return_i + return_j))
            #else:
            #    log_likelihood = torch.log(return_j/(return_i + return_j))
            #print("ll",log_likelihood)
            #cum_log_likelihood += log_likelihood
    return cum_log_likelihood




def random_search(reward_net, demonstrations, num_trials, stdev = 0.1):
    '''hill climbing random search'''
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    best_likelihood = -np.inf
    best_reward = copy.deepcopy(reward_net)
    #create the pairwise rankings for loss calculations
    demo_pairs, preference_labels = create_mcmc_likelihood_data(demonstrations)
    for i in range(num_trials):
        print()
        print("trial", i)
        reward_net_proposal = copy.deepcopy(best_reward)
        #add random noise to weights
        with torch.no_grad():
            for param in reward_net_proposal.parameters():
                param.add_(torch.randn(param.size()).to(device) * stdev)
        #print_traj_returns(reward_net_proposal, demonstrations)
        cum_loglik = calc_pairwise_ranking_loss(reward_net_proposal, demo_pairs, preference_labels)
        print("pair-wise ranking loglik", cum_loglik)
        if cum_loglik > best_likelihood:
            best_likelihood = cum_loglik
            best_reward = copy.deepcopy(reward_net_proposal)
            print("updating best to ", best_likelihood)
        else:
            print("rejecting")
    return best_reward

def generate_feature_counts(demos, reward_net):
    feature_cnts = torch.zeros(len(demos), reward_net.fc2.in_features) #no bias
    for i in range(len(demos)):
        traj = np.array(demos[i])
        traj = torch.from_numpy(traj).float().to(device)
        #print(len(trajectory))
        feature_cnts[i,:] = reward_net.state_features(traj).squeeze().float().to(device)
    return feature_cnts.to(device)

def get_weight_vector(last_layer):
    '''take fc2 layer and return numpy array of weights and bias'''
    linear = last_layer.weight.data
    #print(linear)
    #print(bias)
    with torch.no_grad():
        weights = linear.squeeze().cpu().numpy()
    return weights

def write_weights_likelihood(last_layer, loglik, file_writer):
    if debug:
        print("writing weights")
    #convert last layer to numpy array
    np_weights = get_weight_vector(last_layer)
    for w in np_weights:
        file_writer.write(str(w)+",")
    file_writer.write(str(loglik.item()) + "\n")

def compute_l1(last_layer):
    linear = last_layer.weight.data
    #print(linear)
    #print(bias)
    with torch.no_grad():
        weights = linear.squeeze().cpu().numpy()
    #print("output", np.sum(np.abs(weights)))
    return np.sum(np.abs(weights))

def compute_l2(last_layer):
    linear = last_layer.weight.data
    #print(linear)
    #print(bias)
    with torch.no_grad():
        weights = linear.squeeze().cpu().numpy()
    #print("output", np.sum(np.abs(weights)))
    return np.linalg.norm(weights)


def mcmc_map_search(reward_net, demonstrations, pairwise_prefs, demo_cnts, num_steps, step_stdev, weight_output_filename, weight_init):
    '''run metropolis hastings MCMC and record weights in chain'''
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    demo_pairs, preference_labels = create_mcmc_likelihood_data(demonstrations)

    writer = open(weight_output_filename,'w')

    last_layer = reward_net.fc2
    #set the initial weights to be the signs of the feature counts
    new_data = last_layer.weight.data.cpu().numpy()
    new_data = np.sign(new_data)
    new_data /= np.linalg.norm(new_data)
    print(new_data)
    last_layer.weight.data = torch.from_numpy(new_data).to(device)
    input()
    if weight_init == "randn":
        with torch.no_grad():
            print(last_layer.parameters())
            linear = last_layer.weight.data

            linear.add_(torch.randn(linear.size()).to(device) * step_stdev)
    elif ":" in weight_init:
        print(weight_init.strip().split(':'))
        weight_index, init_weight = weight_init.strip().split(':')
        init_weight = float(init_weight)
        weight_index = int(weight_index)
        print("weight index", weight_index, "init weight", init_weight)
        #initialize with one hot in weight_init position (i.e. initialize in one of the corners of the unit 1-norm sphere)
        with torch.no_grad():
            #get size of weight vector
            num_weights = reward_net.fc2.in_features  #not including the bias weight
            #set up initial weights
            new_linear = torch.zeros(num_weights)
            new_bias = torch.zeros(1)
            if weight_index < num_weights:
                new_linear[weight_index] = init_weight
            else:
                new_bias[0] = init_weight
            #unsqueeze since nn.Linear wants a 2-d tensor for weights
            new_linear = new_linear.unsqueeze(0)
            print("new linear", new_linear)
            print("new bias", new_bias)
            with torch.no_grad():
                #print(last_layer.weight)
                #print(last_layer.bias)
                #print(last_layer.weight.data)
                #print(last_layer.bias.data)
                last_layer.weight.data = new_linear.to(device)
                last_layer.bias.data = new_bias.to(device)
    else:
        print("not a valid weight initialization for MCMC")
        sys.exit()

    #normalize the weight vector to have unit 1-norm...why not unit 2-norm, WCFB won't work without expert...I guess we could do D-REX and estimate
    l2_norm = np.array([compute_l2(last_layer)])

    with torch.no_grad():
        linear = last_layer.weight.data
        print(last_layer.parameters())
        linear.div_(torch.from_numpy(l2_norm).float().to(device))

    if debug:
        print("normalized last layer", compute_l2(last_layer))
        print("weights", get_weight_vector(last_layer))

    #import time
    #start_t = time.time()
    #starting_loglik = calc_pairwise_ranking_loss(reward_net, demo_pairs, preference_labels)
    #end_t = time.time()
    #print("slow likelihood", starting_loglik, "time", 1000*(end_t - start_t))
    #start_t = time.time()
    starting_loglik = calc_linearized_pairwise_ranking_loss(last_layer, pairwise_prefs, demo_cnts)
    #end_t = time.time()
    #print("new fast? likelihood", new_starting_loglik, "time", 1000*(end_t - start_t))
    #print(bunnY)

    map_loglik = starting_loglik
    map_reward = copy.deepcopy(reward_net.fc2)

    cur_reward = copy.deepcopy(reward_net.fc2)
    cur_loglik = starting_loglik



    reject_cnt = 0
    accept_cnt = 0

    for i in range(num_steps):
        if debug:
            print("step", i)
        #take a proposal step
        proposal_reward = copy.deepcopy(cur_reward)
        #add random noise to weights of last layer
        with torch.no_grad():
            for param in proposal_reward.parameters():
                param.add_(torch.randn(param.size()).to(device) * step_stdev)
        l2_norm = np.array([compute_l2(proposal_reward)])
        #normalize the weight vector...
        with torch.no_grad():
            for param in proposal_reward.parameters():
                param.div_(torch.from_numpy(l2_norm).float().to(device))
        if debug:
            print("normalized last layer", compute_l2(proposal_reward))
        #debugging info
        #print_traj_returns(proposal_reward, demonstrations)
        #calculate prob of proposal
        prop_loglik = calc_linearized_pairwise_ranking_loss(proposal_reward, pairwise_prefs, demo_cnts)
        if debug:
            print("proposal loglik", prop_loglik.item())
            print("cur loglik", cur_loglik.item())
        if prop_loglik > cur_loglik:
            #print()
            #accept always
            if debug:
                print("accept")
            accept_cnt += 1
            cur_reward = copy.deepcopy(proposal_reward)
            cur_loglik = prop_loglik

            #check if this is best so far
            if prop_loglik > map_loglik:
                map_loglik = prop_loglik
                map_reward = copy.deepcopy(proposal_reward)
                print()
                print("step", i)

                print("proposal loglik", prop_loglik.item())

                print("updating map to ", prop_loglik)
        else:
            #accept with prob exp(prop_loglik - cur_loglik)
            if np.random.rand() < torch.exp(prop_loglik - cur_loglik).item():
                #print()
                #print("step", i)
                if debug:
                    print("proposal loglik", prop_loglik.item())
                    print("probabilistic accept")
                accept_cnt += 1
                cur_reward = copy.deepcopy(proposal_reward)
                cur_loglik = prop_loglik
            else:
                #reject and stick with cur_reward
                if debug:
                    print("reject")
                reject_cnt += 1

        #save chain of weights
        write_weights_likelihood(cur_reward, cur_loglik, writer)
    print("num rejects", reject_cnt)
    print("num accepts", accept_cnt)
    writer.close()
    return map_reward

In [3]:
env_name ='breakout'
map_reward_model_path = "../mcmc_data/test.params"
seed = 0 #, help="random seed for experiments")
models_dir = "../../learning-rewards-of-learners/learner/models/"#, help="path to directory that contains checkpoint models for demos are stored")
mcmc_step_size = 0.005#', default = 0.005, type=float, help="proposal step is gaussian with zero mean and mcmc_step_size stdev")
pretrained_network = "../pretrained_networks/auxloss/breakout_64_all.params_stripped.params"# ', help='path to pretrained network weights to form \phi(s) using all but last layer')
weight_outputfile="../mcmc_data/test.txt"#', help='filename including path to write the chain to')
debug = False#', help='use fewer demos to speed things up while debugging', action='store_true' )
plot = False#', help='plot out the feature counts over time for demos', action='store_true' )
weight_init = "randn"#', help="defaults to randn, specify integer value to start in a corner of L1-sphere", default="randn")
encoding_dims = 64 #', help="size of latent space", type=int)
mean_path = "/home/dsbrown/Code/deep-bayesian-irl/learned_policies/breakout_linear_mean"
map_path= "/home/dsbrown/Code/deep-bayesian-irl/learned_policies/breakout_linear_map"
render = False#', action='store_true')

In [4]:
env_name = env_name
if env_name == "spaceinvaders":
    env_id = "SpaceInvadersNoFrameskip-v4"
elif env_name == "mspacman":
    env_id = "MsPacmanNoFrameskip-v4"
elif env_name == "videopinball":
    env_id = "VideoPinballNoFrameskip-v4"
elif env_name == "beamrider":
    env_id = "BeamRiderNoFrameskip-v4"
else:
    env_id = env_name[0].upper() + env_name[1:] + "NoFrameskip-v4"

env_type = "atari"
print(env_type)
#set seeds
seed = int(seed)
torch.manual_seed(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

stochastic = True


env = make_vec_env(env_id, 'atari', 1, seed,
                   wrapper_kwargs={
                       'clip_rewards':False,
                       'episode_life':False,
                   })


env = VecFrameStack(env, 4)
agent = PPO2Agent(env, env_type, stochastic)

atari
making vec env

dummy vec




Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use keras.layers.flatten instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
orig_demonstrations, orig_learning_returns, orig_learning_rewards = generate_novice_demos(env, env_name, agent, models_dir)
more_demonstrations, more_learning_returns, more_learning_rewards = generate_mean_map_noop_demos(env, env_name, agent, mean_path, map_path)
print("done")

['00050', '00100', '00150', '00200', '00250', '00300', '00350', '00400', '00450', '00500', '00550', '00600']
checkpoint: 00050, steps: 478, return: 8.0
traj length 478
demo length 0
checkpoint: 00100, steps: 137, return: 0.0
traj length 137
demo length 1
checkpoint: 00150, steps: 264, return: 3.0
traj length 264
demo length 2
checkpoint: 00200, steps: 413, return: 8.0
traj length 413
demo length 3
checkpoint: 00250, steps: 510, return: 10.0
traj length 510
demo length 4
checkpoint: 00300, steps: 731, return: 23.0
traj length 731
demo length 5
checkpoint: 00350, steps: 640, return: 14.0
traj length 640
demo length 6
checkpoint: 00400, steps: 705, return: 16.0
traj length 705
demo length 7
checkpoint: 00450, steps: 725, return: 21.0
traj length 725
demo length 8
checkpoint: 00500, steps: 739, return: 20.0
traj length 739
demo length 9
checkpoint: 00550, steps: 720, return: 17.0
traj length 720
demo length 10
checkpoint: 00600, steps: 815, return: 19.0
traj length 815
demo length 11
check

In [6]:
# print(len(more_demonstrations))
# print(len(more_demonstrations[-3]))
# traj = more_demonstrations[-3]
# traj_rewards = more_learning_rewards[-3]
# for i in range(len(traj)-1,-1,-1):
#     if traj_rewards[i] > 0:
#         indx = i+1
#         print(indx)
#         break
# print(np.sum(traj_rewards[indx:]))

7
1560
1559
0.0


In [ ]:
# #create a snippet for comparison
# good_segment = traj[:indx+1]
# bad_segment = traj[indx:]
# print(len(good_segment))
# print(len(bad_segment))

In [ ]:
def plot_bad_breakout_behavior():
    import matplotlib.pyplot as plt
    print(traj[-1].shape)
    for i in range(2000):
        plt.imshow(traj[-1 - i][0])
    plt.axis('off')
    plt.savefig("../figs/breakout_cant_hit.png")

In [7]:
demonstrations = orig_demonstrations + more_demonstrations
learning_returns = orig_learning_returns + more_learning_returns
learning_rewards = orig_learning_rewards + more_learning_rewards

#sort the demonstrations according to ground truth reward to simulate ranked demos

print([a[0] for a in zip(learning_returns, demonstrations)])
demonstrations = [x for _, x in sorted(zip(learning_returns,demonstrations), key=lambda pair: pair[0])]

sorted_returns = sorted(learning_returns)
print(sorted_returns)

[8.0, 0.0, 3.0, 8.0, 10.0, 23.0, 14.0, 16.0, 21.0, 20.0, 17.0, 19.0, 397.0, 417.0, 431.0, 0.0, 401.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 3.0, 8.0, 8.0, 10.0, 14.0, 16.0, 17.0, 19.0, 20.0, 21.0, 23.0, 397.0, 401.0, 417.0, 431.0]


In [19]:
# Now we download a pretrained network to form \phi(s) the state features where the reward is now w^T \phi(s)
num_mcmc_steps = 100000#', default=2000, type = int, help="number of proposals to generate for MCMC")
mcmc_step_size = 0.001
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
reward_net = EmbeddingNet(encoding_dims)
reward_net.load_state_dict(torch.load(pretrained_network, map_location=device))
#reinitialize last layer
num_features = reward_net.fc2.in_features

print("reward is linear combination of ", num_features, "features")
reward_net.fc2 = nn.Linear(num_features, 1, bias=False) #last layer just outputs the scalar reward = w^T \phi(s)
reward_net.to(device)
#freeze all weights so there are no gradients (we'll manually update the last layer via proposals so no grads required)
for param in reward_net.parameters():
    param.requires_grad = False

#get num_demos by num_features + 1 (bias) numpy array with (un-discounted) feature counts from pretrained network
demo_cnts = generate_feature_counts(demonstrations, reward_net)
print("demo counts")
print(demo_cnts)
#just need index tuples (i,j) denoting j is preferred to i. Assuming all pairwise prefs for now
#check if really better, there might be ties!
pairwise_prefs = []
for i in range(len(demonstrations)):
    for j in range(i+1, len(demonstrations)):
        if sorted_returns[i] < sorted_returns[j]:
            pairwise_prefs.append((i,j))
        else: # they are equal
            print("not using equal prefs", i, j, sorted_returns[i], sorted_returns[j])
            #pairwise_prefs.append((i,j))
            #pairwise_prefs.append((j,i))


#run random search over weights
#best_reward = random_search(reward_net, demonstrations, 40, stdev = 0.01)
best_reward_lastlayer = mcmc_map_search(reward_net, demonstrations, pairwise_prefs, demo_cnts, num_mcmc_steps, mcmc_step_size, weight_outputfile, weight_init)
#turn this into a full network
best_reward = EmbeddingNet(encoding_dims)
#best_reward.fc2 = nn.Linear(num_features, 1, bias=False)
best_reward.load_state_dict(torch.load(pretrained_network, map_location=device))
best_reward.fc2 = best_reward_lastlayer

best_reward.to(device)
#print(best_reward.state_dict())
#save best reward network
#torch.save(best_reward.state_dict(), args.map_reward_model_path)
demo_pairs, preference_labels = create_mcmc_likelihood_data(demonstrations)
print_traj_returns(best_reward, demonstrations)


#add random

reward is linear combination of  64 features
demo counts
tensor([[ 5.3298e+01, -9.1161e+01,  6.6356e+01,  ...,  1.5951e+00,
          1.9192e+01, -4.6382e+01],
        [-4.3560e+03,  8.2327e+03, -4.6915e+03,  ..., -5.7918e+02,
         -1.1762e+03,  3.8790e+03],
        [-2.4695e+03,  4.7959e+03, -2.6950e+03,  ..., -2.2048e+02,
         -6.3044e+02,  2.1376e+03],
        ...,
        [-3.6061e+02,  7.3072e+02, -3.8111e+02,  ..., -3.1567e+01,
         -8.3099e+01,  3.2540e+02],
        [-2.6640e+03,  4.8586e+03, -2.8732e+03,  ..., -1.6409e+02,
         -7.1788e+02,  2.2514e+03],
        [-1.3175e+03,  2.4406e+03, -1.4275e+03,  ..., -6.5463e+01,
         -3.5764e+02,  1.1275e+03]], device='cuda:0')
not using equal prefs 0 1 0.0 0.0
not using equal prefs 0 2 0.0 0.0
not using equal prefs 0 3 0.0 0.0
not using equal prefs 1 2 0.0 0.0
not using equal prefs 1 3 0.0 0.0
not using equal prefs 2 3 0.0 0.0
not using equal prefs 5 6 8.0 8.0
[[ 0.125 -0.125  0.125  0.125 -0.125 -0.125  0.125  0.12


step 178
proposal loglik -21049.787109375
updating map to  tensor(-21049.7871)

step 179
proposal loglik -20924.35546875
updating map to  tensor(-20924.3555)

step 180
proposal loglik -20532.52734375
updating map to  tensor(-20532.5273)

step 181
proposal loglik -20368.9453125
updating map to  tensor(-20368.9453)

step 184
proposal loglik -19104.744140625
updating map to  tensor(-19104.7441)

step 186
proposal loglik -17637.576171875
updating map to  tensor(-17637.5762)

step 187
proposal loglik -16725.70703125
updating map to  tensor(-16725.7070)

step 188
proposal loglik -16496.845703125
updating map to  tensor(-16496.8457)

step 191
proposal loglik -15201.0322265625
updating map to  tensor(-15201.0322)

step 192
proposal loglik -14161.365234375
updating map to  tensor(-14161.3652)

step 193
proposal loglik -13396.5693359375
updating map to  tensor(-13396.5693)

step 195
proposal loglik -13302.296875
updating map to  tensor(-13302.2969)

step 198
proposal loglik -12348.3486328125
up


step 869
proposal loglik -39.00806427001953
updating map to  tensor(-39.0081)

step 1001
proposal loglik -38.72477722167969
updating map to  tensor(-38.7248)

step 1472
proposal loglik -36.786277770996094
updating map to  tensor(-36.7863)

step 1485
proposal loglik -36.74752426147461
updating map to  tensor(-36.7475)

step 1552
proposal loglik -36.29877471923828
updating map to  tensor(-36.2988)

step 1592
proposal loglik -34.68990707397461
updating map to  tensor(-34.6899)

step 1640
proposal loglik -34.4594612121582
updating map to  tensor(-34.4595)

step 1663
proposal loglik -33.729515075683594
updating map to  tensor(-33.7295)

step 1892
proposal loglik -33.410186767578125
updating map to  tensor(-33.4102)

step 1931
proposal loglik -33.348838806152344
updating map to  tensor(-33.3488)

step 2138
proposal loglik -32.33916473388672
updating map to  tensor(-32.3392)

step 2176
proposal loglik -32.312355041503906
updating map to  tensor(-32.3124)

step 2181
proposal loglik -31.100795